In [ ]:
import numpy as np

from skimage import io , data , exposure ,color

import matplotlib.pyplot as plt

In [ ]:
from prompt_toolkit.shortcuts.progress_bar.formatters import Percentage
class Image:
  import numpy as np
  def __init__(self,data:np.ndarray):
    self.data=data
    self.height= data.shape[0]
    self.width=data.shape[1]
    self.datatype=data.dtype


  def is_grayscale(self):
    return len(self.data.shape)==2


  def imshow(self):
    import matplotlib.pyplot as plt
    if (self.is_grayscale()):
      plt.imshow(self.data,cmap="gray")


  def histogram(self,intensity_resolution=8,normalize=False):
    import numpy as np
    if self.is_grayscale():
      bins=np.arange(0,2**intensity_resolution,1)
      hist=np.zeros_like(bins)
      for i in range(self.height):
        for j in range(self.width):
          hist[self.data[i,j]] = hist[self.data[i,j]]*((2**intensity_resolution)/256)
      if (normalize):
          hist=hist/(self.height*self.width)
    return bins, hist
  def plot_histogram(self, nbins=256, normalize=False, plot_cdf=True):
    hist,bins_hist=exposure.histogram(self.data.ravel(),nbins=nbins,normalize=True)
    plt.plot(bins_hist, hist,'k')
    plt.xlabel("pixel value")
    if normalize:
      plt.ylabel('probabilty')
    else:
      plt.ylabel('count')
    xmax=1 if self.data.max()<=1 else 255
    plt.xlim([0,xmax])
    if plot_cdf:
      cdf,bins_cdf=exposure.cumulative_distribution(self.data.ravel(),nbins=nbins)
      plt.twinx()
      plt.plot(bins_cdf,cdf,'r',lw=3)
      plt.ylabel ('Percentage')
  def plot_img_and_hist(self, nbins=256, normalize=False, plot_cdf=True ):
    plt.subplot(2,1,1)
    io.imshow(self.data)
    plt.axis('off')
    plt.title('image')
    plt.subplot(2,1,2)
    self.plot_histogram(nbins=nbins,normalize=normalize,plot_cdf=plot_cdf)
    plt.title('histogram')
  # def plot_histogram(self,intensity_resolution=8,normalize=False):
  #   import matplotlib.pyplot as plt
  #   bins, hist=self.histogram(intensity_resolution=intensity_resolution,normalize=normalize)
  #   plt.plot(bins, hist)
  def cumulative_histogram(self,intensity_resolution=8,normalize=False):
    bins, hist=self.histogram(intensity_resolution=intensity_resolution,normalize=normalize)
    for i in range(1,len(hist)):
      hist[i]=hist[i-1]+hist[i]
    return bins,hist


  def plot_cumulative_histogram(self,intensity_resolution=8,normalize=False):
    import matplotlib.pyplot as plt
    bins, hist=self.cumulative_histogram(intensity_resolution=intensity_resolution,normalize=normalize)
    plt.plot(bins, hist)


  def contrast_streching(self,intensity_resolution=8,normalize=False):
    import numpy as np
    r=self.data/255
    h=np.max(r)
    l=np.min(r)
    s=(((1/(h-l))*(r-l))*255).astype(np.uint8)
    return Image(s)


  def contrast_streching2(self,intensity_resolution=8,normalize=False):
    import numpy as np
    s_I=Image(self.data/2**intensity_resolution)
    H=s_I.data.max()
    L=s_I.data.min()
    s_I.data=np.clip((s_I.data-L)*(1/H-L),0,1)*2**intensity_resolution
    s_I.data=s_I.data.astype(np.uint8)
    return Image(s_I)


  def power_transform(self,gamma):
    import numpy as np
    r=self.data/255
    r=r**gamma
    c=np.max(r)
    s=r/c
    return Image(s)
  def plot_histogram_color(self, nbins=256, normalize=False, plot_cdf=True):
    colour=['r','g','b']
    for i in range(3):
      hist,bins_hist=exposure.histogram(self.data[:,:,i].ravel(),nbins=nbins,normalize=True)
      plt.plot(bins_hist, hist,colour[i])
    plt.xlabel("pixel value")
    if normalize:
      plt.ylabel('probabilty')
    else:
      plt.ylabel('count')
    xmax=1 if self.data.max()<=1 else 255
    plt.xlim([0,xmax])
    plt.legend(['red channel','green channel','blue channel'])
    # if plot_cdf:
    #   cdf,bins_cdf=exposure.cumulative_distribution(self.data.ravel(),nbins=nbins)
    #   plt.twinx()
    #   plt.plot(bins_cdf,cdf,'r',lw=3)
    #   plt.ylabel ('Percentage')
  def plot_img_and_hist_color(self, nbins=256, normalize=False ):
      plt.subplot(2,1,1)
      io.imshow(self.data)
      plt.axis('off')
      plt.title('image')
      plt.subplot(2,1,2)
      self.plot_histogram_color(nbins=nbins,normalize=normalize)
      plt.title('histogram')
  def ad_hist(self,nbins=256,window=(30,30),clip_limit=0.01,normal_he=True):
    I_gray=self.data/255
    if normal_he:
      data_eq= exposure.equalize_hist(I_gray,nbins=nbins)
    else:
      data_eq=exposure.equalize_adapthist(I_gray,kernel_size=window, clip_limit=clip_limit,nbins=nbins)
    return data_eq
        # I_gray_norm=exposure.equalize_adapthist(I_gray,kernel_size=(50,50),clip_limit=0.04,nbins=256)
        # plt.subplot(2,2,1)
        # io.imshow(I_gray)
        # plt.subplot(2,2,2)
        # io.imshow(I_gray_norm)
        # plt.subplot(2,2,3)
        # x,y=exposure.cumulative_distribution(I_gray.ravel(),nbins=256)
        # plt.plot(x,y)
        # plt.subplot(2,2,4)
        # x1,y1=exposure.cumulative_distribution(I_gray_norm.ravel(),nbins=256)
        # plt.plot(x1,y1)

In [ ]:
I_gray=data.cat()
img_gray=Image(I_gray)
print(img_gray.plot_histogram())
img_gray.plot_img_and_hist_color()

In [ ]:
I=data.camera()
x=Image(I)
I_eq=Image(x.ad_hist(normal_he=False,clip_limit=0))
I_eq.plot_img_and_hist()

In [ ]:
I_color=data.cat()
I_color_lab=color.rgb2lab(I_color)

I_color_lab[:,:,0]=(I_color_lab[:,:,0]/100)
I_color_lab_eq= exposure.equalize_adapthist(I_color_lab[:,:,0],kernel_size=(100,100),clip_limit=0.01,nbins=256)
img_lab=Image(I_color_lab_eq)
img_lab.plot_histogram()

In [ ]:
I_color_lab[:,:,0] =I_color_lab_eq*100
I_color_recon=color.lab2rgb(I_color_lab)
io.imshow(I_color_recon)

In [ ]:
a=data.grass()
a=Image(a)
a.plot_img_and_hist()
c=a.contrast_streching()
c.imshow()
c.plot_img_and_hist()